In [1]:
import hexathello.aiPlayers as aiPlayers
import hexathello.AutoPlayer as autoPlayer
import hexathello.Engine as engine
import hexathello.history as history
import hexathello.jable as jable
import hexathello.printing as printing

import numpy as np

from os import path

# -- Settings
game_size: int = 5
player_count: int = 2

In [2]:
# We want to train a KerasHexAgent. To do this, we use data we created in `quickstart_recording_data.ipynb`

history_dir: str = path.join(
    'data',
    'history',
    'examples'
)
assert path.isdir(
    history_dir
)

baseline_data_path: str = path.join(
    history_dir,
    'greendom_size-{}_players-{}.json'.format(
        game_size, player_count
    )
)
   

In [3]:
# Read the data from disk to learn from
history_fromDisk: jable.JyFrame = jable.read_file(
    baseline_data_path
)

# Decode the state, option, and play vectors from integers to numpy arrays
history_decoded: jable.JyFrame = history.history_fromInt(
    history_fromDisk
)

assert len( history_decoded ) >= 20000
    
# Make it PoV 0 to appropriately learn
povHistory: jable.JyFrame = history.povHistory_from_literalHistory(
    history_decoded
)

del history_decoded
del history_fromDisk

In [4]:
import os
import math
import tensorflow as tf

input_size: int = len(povHistory[0, 'board_state'])
output_size: int = len(povHistory[0, 'player_action'])
save_dir = path.join("data","ai","nuerons_per_layer")
os.makedirs(save_dir, exist_ok=True)

for i in range(7):
    # using 720 neurons and go to 7 layers
    num_layers = i + 1
    max_neurons = 720
    neurons_per_layer = math.floor(max_neurons / num_layers)

    print(f"Training {num_layers} layers with {neurons_per_layer} neurons each")

    ai_keras_id = "{}x{}_total{}_size{}_players{}_v1".format(
        num_layers, neurons_per_layer, num_layers * neurons_per_layer, game_size, player_count
    )

    ai_keras_path: str = path.join(
        'data',
        'ai',
        'vary_dimensions',
        '{}.keras'.format(ai_keras_id)
    )

    os.makedirs(path.dirname(ai_keras_path), exist_ok=True)

    if path.isfile(ai_keras_path):
        brain_model = tf.keras.models.load_model(ai_keras_path)
    else:
        brain_input = tf.keras.layers.Input(shape=(input_size,), name='keras_tensor')
        x = brain_input

        for _ in range(num_layers):
            x = tf.keras.layers.Dense(neurons_per_layer, activation='relu')(x)

        brain_output = tf.keras.layers.Dense(output_size, activation='sigmoid')(x)

        brain_model = tf.keras.Model(inputs=brain_input, outputs=brain_output)
        brain_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), # making it higher 2-5x -- changes params faster for new training
            loss=tf.keras.losses.CategoricalCrossentropy()
        )

    ai_keras = aiPlayers.KerasHexAgent(
        size=game_size,
        player_count=player_count,
        brain=brain_model,
        player_id=None,
        ai_id=ai_keras_id
    )

    ai_keras_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=ai_keras_path,
        monitor='loss',
        mode='min',
        save_best_only=True
    )

    ai_keras.train(
        game_history=povHistory,
        epochs=10,
        callbacks=[ai_keras_checkpoint_callback]
    )

    if path.isfile(ai_keras_path):
        print(f"Model saved successfully at: {ai_keras_path}")
    else:
        print("Model not saved. Check callback settings or folder path.")
    ai_keras.brain.summary()


Training 1 layers with 720 neurons each
Epoch 1/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1.3096
Epoch 2/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.8311
Epoch 3/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.6857
Epoch 4/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.5704
Epoch 5/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.4916
Epoch 6/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.4148
Epoch 7/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3589
Epoch 8/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3002
Epoch 9/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2900
Epoch 10/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2376
Model saved successfully at: data\ai\vary_dimensions\1x720_total720_size5_players2_v1.keras


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ keras_tensor (InputLayer)            │ (None, 122)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 720)                 │          88,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 61)                  │          43,981 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 397,625 (1.52 MB)

 Trainable params: 132,541 (517.74 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ keras_tensor (InputLayer)            │ (None, 122)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 102)                 │          12,546 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 102)                 │          10,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 102)                 │          10,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 102)                 │          10,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 102)                 │          10,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 102)                 │          10,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 102)                 │          10,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 61)                  │           6,283 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 245,597 (959.37 KB)

 Trainable params: 81,865 (319.79 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 163,732 (639.58 KB)